In [1]:
import pandas as pd
import numpy as np
import random
import re
from datetime import datetime
%matplotlib inline

# Making the Sim

In [2]:
#gamelogs contains a players stats for a specific game
#gameresults contains which teams played in a specific game and their stats
gamelogs = pd.read_csv("PlayerGameLog18.csv.csv")
gameresults = pd.read_csv("TeamGamesLogsDF18.csv.csv")

In [3]:
gamelogs['GAME_DATE'] = pd.to_datetime(gamelogs['GAME_DATE'])
gamelogs.set_index('GAME_DATE', inplace=True)

In [4]:
time = gameresults
time['GAME_DATE'] = pd.to_datetime(time['GAME_DATE'])
time.set_index('GAME_DATE', inplace=True)

In [5]:
pretradegames = time.loc['10/16/2018':'2/8/2019']
posttradegames = time.loc['2/9/2019':]

In [6]:
#example of what is contained in the GAME_ID column
posttradegames['GAME_ID'].iloc[1]

21800829

In [7]:
gameid_list = gameresults['GAME_ID'].drop_duplicates().to_list()

In [8]:
len(gameid_list)

1230

In [10]:
#just checking how to properly get the team abbreviation from a certain game_id
#game_id just represents the game that was played and what teams played
bos_phi = gameresults[gameresults['GAME_ID'] == gameid_list[0]]['TEAM_ABBREVIATION'].to_list()

In [11]:
bos_phi[1]

'PHI'

In [12]:
#setting the two rosters, one from before the trade deadline and one after the trade deadline
pretradelog = gamelogs.loc['10/16/2018':'11/12/2018']
posttradelog = gamelogs.loc['2/8/2019':]

In [13]:
def set_team(matchup):
    '''
    Gets first team from matchup column from a game.
    '''
    team = re.match(r"([A-z]+) (vs.|@)", matchup)
    return team.groups(1)[0]

In [14]:
#gamelogs_with_team = gamelogs.copy()
#gamelogs_with_team["TEAM"] = gamelogs_with_team["MATCHUP"].map(set_team)

In [15]:
#Beginning of the season roster
gamelogs_with_team = pretradelog.copy()
gamelogs_with_team["TEAM"] = gamelogs_with_team["MATCHUP"].map(set_team)

#Post trade-deadline roster
gamelogs_with_team2 = posttradelog.copy()
gamelogs_with_team2["TEAM"] = gamelogs_with_team2["MATCHUP"].map(set_team)

In [16]:
#getting all 30 teams
team_list =  gamelogs_with_team['TEAM'].drop_duplicates().to_list()
type(team_list)

list

In [18]:
#making sure there is 30 teams
len(team_list)

30

In [19]:
def get_roster_ids(team):
    '''
    Takes in team abreviation, returns list of player IDs. Before trade deadline.
    '''
    if type(team) == int:
        team = gameresults[gameresults["TEAM_ID"]==team].iloc[0]["TEAM_ABBREVIATION"]
    playerids = gamelogs_with_team[gamelogs_with_team["TEAM"]==team]["Player_ID"].drop_duplicates()
    return playerids.values

def get_roster_ids2(team):
    '''
    Takes in team abreviation, returns list of player IDs. After trade deadline.
    '''
    if type(team) == int:
        team = gameresults[gameresults["TEAM_ID"]==team].iloc[0]["TEAM_ABBREVIATION"]
    playerids = gamelogs_with_team2[gamelogs_with_team2["TEAM"]==team]["Player_ID"].drop_duplicates()
    return playerids.values

In [21]:
#making sure above function returns what we expect
hou = get_roster_ids("HOU")
hou

array([ 101108,  101123, 1627737, 1627753, 1628392, 1629109,  200782,
        201569,  201935,  203487,  203516,  203991,    2546], dtype=int64)

In [22]:
#getting averages and standard deviation for stat distributions
#checked on stats_distribution_notebook, were approx. normal
group = gamelogs.groupby(by='Player_ID')['PTS'].agg(['mean','std'])
g_ass = gamelogs.groupby(by='Player_ID')['AST'].agg(['mean','std'])
g_reb = gamelogs.groupby(by='Player_ID')['REB'].agg(['mean','std'])
g_tov = gamelogs.groupby(by='Player_ID')['TOV'].agg(['mean','std'])
g_stl = gamelogs.groupby(by='Player_ID')['STL'].agg(['mean','std'])
g_blk = gamelogs.groupby(by='Player_ID')['BLK'].agg(['mean','std'])

group.reset_index(inplace=True)
g_ass.reset_index(inplace=True)
g_reb.reset_index(inplace=True)
g_tov.reset_index(inplace=True)
g_stl.reset_index(inplace=True)
g_blk.reset_index(inplace=True)

In [23]:
#fill in 0 for players that didn't play
group.fillna(0,inplace=True)
g_ass.fillna(0, inplace=True)
g_reb.fillna(0, inplace=True)
g_tov.fillna(0, inplace=True)
g_stl.fillna(0, inplace=True)
g_blk.fillna(0, inplace=True)

In [24]:
def sim_player(playerID, numSims=1):
    '''
    Takes in a player ID and calculates their points "scored" in a game. AKA their contribution to their team winning
    by sampling from their stats and combining for their contribution.
    '''
    player_games = gamelogs[gamelogs["Player_ID"]==playerID].drop(columns=["SEASON_ID", "Player_ID", "Game_ID", "MATCHUP", "WL", "VIDEO_AVAILABLE"])
    simmed_games = []
    sim = 1
    while sim <= numSims:
        randGame = random.randint(1,83)
        if randGame > len(player_games):
            #player_games.iloc[len(player_games)]=np.zeros(20)
            return 0
        else:
            pts = (round(int(np.random.normal(group[group['Player_ID']==playerID]['mean'],group[group['Player_ID']==playerID]['std'],1))))
            reb = (round(int(np.random.normal(g_reb[g_reb['Player_ID']==playerID]['mean'],g_reb[g_reb['Player_ID']==playerID]['std'],1))))
            ass = (round(int(np.random.normal(g_ass[g_ass['Player_ID']==playerID]['mean'],g_ass[g_ass['Player_ID']==playerID]['std'],1))))
            stl = (round(int(np.random.normal(g_stl[g_stl['Player_ID']==playerID]['mean'],g_stl[g_stl['Player_ID']==playerID]['std'],1))))
            blk = (round(int(np.random.normal(g_blk[g_blk['Player_ID']==playerID]['mean'],g_blk[g_blk['Player_ID']==playerID]['std'],1))))
            tov = (round(int(np.random.normal(g_tov[g_tov['Player_ID']==playerID]['mean'],g_tov[g_tov['Player_ID']==playerID]['std'],1))))
            score = pts + round(1.7*reb) + round(2*ass) + round(9*stl)+ round(6*blk)-round(5*tov)
            if score >= 0:
                return score
            else:
                return 0
        sim += 1
#https://fivethirtyeight.com/features/the-hidden-value-of-the-nba-steal/
#article goes into detail about the contribution aspect

In [25]:
def sim_team(playerIDs, numSims=1):
    '''
    Takes in player IDs to pass through to player_sims function.
    Returns points scored. Can keep track of a list of player IDs
    which is a team.
    '''
    sims_raw = []
    for playerID in playerIDs:
        player_sims = sim_player(playerID, numSims)
        sims_raw.append(player_sims)
    return sims_raw

In [26]:
#checking to make sure it works
sum(sim_team(get_roster_ids('HOU'),1))

264

In [27]:
#another check
team1_sims = sim_team(get_roster_ids("CHA"), 1)

team1_sims

[0, 22, 20, 15, 6, 17, 0, 0, 10, 51, 0, 17, 42, 0]

In [28]:
#before trade deadline
def sim_games(team1, team2, numSims=1):
    '''
    Takes in two teams and passes them through the sim team function 
    and returns how many games each team won. Uses get_roster_ids to get player
    IDs to pass through.
    '''
    team1_roster = get_roster_ids(team1)
    team2_roster = get_roster_ids(team2)
    results = [0,0]
    for game in range(0, numSims):
        team1_sims = sim_team(team1_roster, 1)
        team2_sims = sim_team(team2_roster, 1)
        if sum(team1_sims) > sum(team2_sims):
            results[0] += 1
        else:
            results[1] += 1
    return results

#after trade deadline
def sim_games2(team1, team2, numSims=1):
    '''
    Takes in two teams and passes them through the sim team function 
    and returns how many games each team won.
    '''
    team1_roster = get_roster_ids2(team1)
    team2_roster = get_roster_ids2(team2)
    results = [0,0]
    for game in range(0, numSims):
        team1_sims = sim_team(team1_roster, 1)
        team2_sims = sim_team(team2_roster, 1)
        if sum(team1_sims) > sum(team2_sims):
            results[0] += 1
        else:
            results[1] += 1
    return results

In [30]:
#checking to make sure it works, HOU should beat CLE handily
sim_games2("HOU", "CLE", 10)

[10, 0]

In [31]:
def get_game(gameid):
    '''
    Takes in a game ID and returns which two teams played in that game.
    '''
    teams = gameresults[gameresults['GAME_ID'] == gameid_list[gameid]]['TEAM_ABBREVIATION'].to_list()
    return teams[0], teams[1]   

In [32]:
#checking to make sure it works
get_game(1)

('GSW', 'OKC')

In [33]:
def sim_season(numSimss=1):
    '''
    Takes in how many seasons you want to run and returns a dataframe
    of the different seasons and each team's number of wins in that season.
    '''
    team_wins = pd.DataFrame(columns=[team_list])
    team_wins = team_wins.append(pd.Series(name='Win_Count'))
    team_wins.fillna(0,inplace=True)
    
    for games in range(len(gameid_list)):
        wins = []
        if gameid_list[games] == pretradegames['GAME_ID'].iloc[games]:
            wins = sim_games(get_game(games)[0],get_game(games)[1],numSimss)
            if wins[0] > wins[1]:
                team_wins[get_game(games)[0]] +=1
            else:
                team_wins[get_game(games)[1]] +=1
        else:
            wins = sim_games2(get_game(games)[0],get_game(games)[1],numSimss)
            if wins[0] > wins[1]:
                team_wins[get_game(games)[0]] +=1
            else:
                team_wins[get_game(games)[1]] +=1
    return team_wins

# Running the sim

In [ ]:
%%time
#running one sim
sim_wins = sim_season(1)

In [ ]:
pivotwins = sim_wins.transpose()

In [ ]:
pivotwins.sort_values(by='Win_Count', ascending=False)

In [354]:
import functools 

In [353]:
#checking how to merge multiple simulated seasons together
df_merged = functools.reduce(lambda  left,right: pd.merge(left,right,on=['level_0'],
                                            how='outer'), listwins)

NameError: name 'listwins' is not defined

In [ ]:
#creating a list for sim names
sim_list = []
for i in range(100):
    sim_list.append('sim'+str(i))

In [ ]:
#running the simulation 100 times aka 100 seasons
filled_sim_list = []
for simulation in sim_list:
    simulation = sim_season(1)
    simulation = simulation.transpose()
    simulation = simulation.reset_index()
    filled_sim_list.append(simulation)

In [ ]:
#merging into one dataframe
simulation100 = functools.reduce(lambda  left,right: pd.merge(left,right,on=['level_0'],
                                            how='outer'), filled_sim_list)

In [ ]:
#checking
simulation100

In [ ]:
#writing to CSV
simulation100.to_csv('simulation_results.csv')

# Modifying for All Star Analysis

In [67]:
#list of all-star players, ASG is guards, ASW is wings, and ASF is Frontcourts or bigs 
#list was obtained from the all-nba awards and all-star awards found on NBA.com
ASG = [201935,202681,201939,202689,203078,203081,201566]
ASW = [201142,2544,202695,202331,203114,203507,1627732]
ASF = [203999,203954,203497,1626157,200746,202696,203076]

[48, 52, 55, 58, 42, 63, 35]

In [68]:
def sim_all_star_games(team1, team2, numSims=1):
    '''
    Takes in two teams and passes them through the sim team function 
    and returns how many games each team won. 
    '''
    team1_roster = team1
    team2_roster = team2
    results = [0,0]
    for game in range(0, numSims):
        team1_sims = sim_team(team1_roster, 1)
        team2_sims = sim_team(team2_roster, 1)
        if sum(team1_sims) > sum(team2_sims):
            results[0] += 1
        else:
            results[1] += 1
    return results

In [ ]:
#simulating the all-star teams against each other

In [70]:
sim_all_star_games(ASG,ASW,1000)
#guards beating wings

[579, 421]

In [71]:
sim_all_star_games(ASG,ASF,1000)
#Guards losing to bigs

[419, 581]

In [72]:
sim_all_star_games(ASF,ASW,1000)
#Bigs beating wings

[654, 346]

# Taking out the "injuries" aspect to see if that changes results

In [289]:
def sim_player_without_injuries(playerID, numSims=1):
    '''
    Takes in a player ID and calculates there points scored in a game. 
    '''
    player_games = gamelogs[gamelogs["Player_ID"]==playerID].drop(columns=["SEASON_ID", "Player_ID", "Game_ID", "MATCHUP", "WL", "VIDEO_AVAILABLE"])
    simmed_games = []
    sim = 1
    while sim <= numSims:
        #modified this part to insure that they always play
        randGame = random.randint(1,3)
        if randGame > len(player_games):
            #player_games.iloc[len(player_games)]=np.zeros(20)
            return 0
        else:
            pts = (round(int(np.random.normal(group[group['Player_ID']==playerID]['mean'],group[group['Player_ID']==playerID]['std'],1))))
            reb = (round(int(np.random.normal(g_reb[g_reb['Player_ID']==playerID]['mean'],g_reb[g_reb['Player_ID']==playerID]['std'],1))))
            ass = (round(int(np.random.normal(g_ass[g_ass['Player_ID']==playerID]['mean'],g_ass[g_ass['Player_ID']==playerID]['std'],1))))
            stl = (round(int(np.random.normal(g_stl[g_stl['Player_ID']==playerID]['mean'],g_stl[g_stl['Player_ID']==playerID]['std'],1))))
            blk = (round(int(np.random.normal(g_blk[g_blk['Player_ID']==playerID]['mean'],g_blk[g_blk['Player_ID']==playerID]['std'],1))))
            tov = (round(int(np.random.normal(g_tov[g_tov['Player_ID']==playerID]['mean'],g_tov[g_tov['Player_ID']==playerID]['std'],1))))
            score = pts + (1.7*reb) + (2*ass) + (9*stl)+ (6*blk)-(5*tov)
            if score >= 0:
                return score
            else:
                return 0
        sim += 1

In [290]:
def sim_team_all_star_without_injuries(playerIDs, numSims=1):
    '''
    Takes in player IDs to pass through to player_sims function.
    Returns points scored. Can keep track of a list of player IDs
    which is a team.
    '''
    sims_raw = []
    for playerID in playerIDs:
        player_sims = sim_player_without_injuries(playerID, numSims)
        sims_raw.append(player_sims)
    return sims_raw

In [291]:
def sim_all_star_games_without_injuries(team1, team2, numSims=1):
    '''
    Takes in two teams and passes them through the sim team function 
    and returns how many games each team won. 
    '''
    team1_roster = team1
    team2_roster = team2
    results = [0,0]
    for game in range(0, numSims):
        team1_sims = sim_team_all_star_without_injuries(team1_roster, 1)
        team2_sims = sim_team_all_star_without_injuries(team2_roster, 1)
        if sum(team1_sims) > sum(team2_sims):
            results[0] += 1
        else:
            results[1] += 1
    return results

In [292]:
sim_all_star_games_without_injuries(ASG,ASW,1000)

[489, 511]

In [293]:
sim_all_star_games_without_injuries(ASG,ASF,1000)

[317, 683]

In [294]:
sim_all_star_games_without_injuries(ASF,ASW,1000)

[684, 316]

In [ ]:
#frontcourt still wins, guards and wings about even

# Average All-Star with average team 

In [107]:
def position_mapper(x):
    '''
    maps g to allstar guards
    maps w to allstar wings
    maps f to allstar frontcourt players
    '''
    if x in ASG:
        return 'G'
    elif x in ASW:
        return 'W'
    elif x in ASF:
        return 'F'
    else:
        return 'N'

In [119]:
all_stars_mapped = gamelogs.copy()

In [120]:
all_stars_mapped['position'] = all_stars_mapped['Player_ID'].map(position_mapper)

In [148]:
#finding the average stat for the all-star players by class
#will have all-star guard(g), wing(w), frontcourt(f), and normal players(n)
all_star_avg = all_stars_mapped.groupby(by='position')[['PTS','REB','AST','TOV','STL','BLK']].mean()
all_star_std = all_stars_mapped.groupby(by='position')[['PTS','REB','AST','TOV','STL','BLK']].std()

In [151]:
all_star_std.rename(columns={'PTS': 'pts_std', 'REB': 'reb_std','AST':'ast_std','TOV':'tov_std','STL':'stl_std','BLK':'blk_std'},inplace=True)

In [153]:
#all_star_avg.columns.set_levels(['pts_average','pts_std','reb_average','reb_std','ast_average','ast_std','tov_average','tov_std','stl_average','stl_std','blk_average','blk_std'],level=1,inplace=True)

In [157]:
asm = all_star_avg.merge(all_star_std, left_on='position',right_on='position')

In [158]:
#calcuating their contribution
asm['contribution']=asm['PTS'] + 1.7*asm['REB'] + 2*asm['AST'] + 9*asm['TOV'] + 6*asm['STL'] - 5*asm['BLK']

In [161]:
#can see that guards have best contribution but why do the frontcourt win?
asm.sort_values(by='contribution',ascending=False)

,PTS,REB,AST,TOV,STL,BLK,pts_std,reb_std,ast_std,tov_std,stl_std,blk_std,contribution
position,,,,,,,,,,,,,
G,26.813559,5.973635,6.926554,3.244821,1.517891,0.519774,9.831653,3.391897,3.511742,2.010432,1.367121,0.727547,86.533710
W,24.146586,8.202811,5.608434,2.939759,1.385542,0.708835,8.449435,3.825288,2.925172,1.781976,1.267117,0.977391,80.535141
F,21.913295,11.766859,3.763006,2.425819,0.961464,1.579961,8.491092,4.343101,2.705582,1.699928,1.010781,1.396653,69.144316
N,9.609294,3.980898,2.120270,1.174887,0.683582,0.437258,7.512315,3.326570,2.328597,1.291957,0.929994,0.801636,33.106549


# Checking Why F are the best

In [164]:
games_played = all_stars_mapped.groupby(by='position')[['Game_ID']].count()

In [168]:
#ignore N for this
games_played['Percent Played']=(games_played['Game_ID']/(7*82))*100

In [169]:
#wings played the least but why did fowards win? More reliable? I.E. lower standard deviations?
games_played

,Game_ID,Percent Played
position,,
F,519,90.418118
G,531,92.508711
N,24553,4277.526132
W,498,86.759582


In [297]:
def sim_player_avg_as(playerID, numSims=1):
    '''
    Takes in a player ID and calculates there points scored in a game. This is for the average all star stats.
    Not accounting for injuries.
    '''
    sim = 1
    while sim <= numSims:
        pts = (round(int(np.random.normal(asm[asm.index ==playerID]['PTS'],asm[asm.index==playerID]['pts_std'],1))))
        reb = (round(int(np.random.normal(asm[asm.index ==playerID]['REB'],asm[asm.index==playerID]['reb_std'],1))))
        ass = (round(int(np.random.normal(asm[asm.index ==playerID]['AST'],asm[asm.index==playerID]['ast_std'],1))))
        stl = (round(int(np.random.normal(asm[asm.index ==playerID]['STL'],asm[asm.index==playerID]['stl_std'],1))))
        blk = (round(int(np.random.normal(asm[asm.index ==playerID]['BLK'],asm[asm.index==playerID]['blk_std'],1))))
        tov = (round(int(np.random.normal(asm[asm.index ==playerID]['TOV'],asm[asm.index==playerID]['tov_std'],1))))
        score = pts + round(1.7*reb) + round(2*ass) + round(9*stl) + round(6*blk) - round(5*tov)
        if score >= 0:
            return score
        else:
            return 0
        sim += 1

In [302]:
#simulating total contribution with their standard deviation per game.
g_total=[]
w_total=[]
f_total=[]
n_total=[]
for i in range(10000):
    g_contribution = sim_player_avg_as('G',1)
    w_contribution = sim_player_avg_as('W',1)
    f_contribution = sim_player_avg_as('F',1)
    n_contribution = sim_player_avg_as('N',1)
    g_total.append(g_contribution)
    w_total.append(w_contribution)
    f_total.append(f_contribution)
    n_total.append(n_contribution)

In [303]:
import statistics
print('guards:', sum(g_total), 'std', statistics.stdev(g_total))
print('wings:', sum(w_total), 'std', statistics.stdev(w_total))
print('frontcourt:', sum(f_total), 'std',statistics.stdev(f_total))
print('normies:', sum(n_total),'std',statistics.stdev(n_total))

guards: 464905 std 19.785738735152893
wings: 465382 std 18.39217681452627
frontcourt: 504846 std 18.03163544517476
normies: 200812 std 12.673889273105186


In [ ]:
#bigs have lower standard dev.
#also maybe too much emphasis on blocks?
#find better distribution for blocks?

# Removing MVP candidates to see who had the biggest impact on team wins

In [322]:
#list of top players recieving MVP votes. If teams had multiple players, the one with the most was selected.
#https://www.basketball-reference.com/awards/awards_2019.html

In [325]:
def sim_player_mvp(playerID, numSims=1):
    '''
    Takes in a player ID and calculates there points scored in a game. Will not select the chosen MVP candidate.
    '''
    player_games = gamelogs[gamelogs["Player_ID"]==playerID].drop(columns=["SEASON_ID", "Player_ID", "Game_ID", "MATCHUP", "WL", "VIDEO_AVAILABLE"])
    simmed_games = []
    sim = 1
    while sim <= numSims:
        randGame = random.randint(1,83)
        if randGame > len(player_games) or player == playerID:
            #player_games.iloc[len(player_games)]=np.zeros(20)
            return 0
        else:
            pts = (round(int(np.random.normal(group[group['Player_ID']==playerID]['mean'],group[group['Player_ID']==playerID]['std'],1))))
            reb = (round(int(np.random.normal(g_reb[g_reb['Player_ID']==playerID]['mean'],g_reb[g_reb['Player_ID']==playerID]['std'],1))))
            ass = (round(int(np.random.normal(g_ass[g_ass['Player_ID']==playerID]['mean'],g_ass[g_ass['Player_ID']==playerID]['std'],1))))
            stl = (round(int(np.random.normal(g_stl[g_stl['Player_ID']==playerID]['mean'],g_stl[g_stl['Player_ID']==playerID]['std'],1))))
            blk = (round(int(np.random.normal(g_blk[g_blk['Player_ID']==playerID]['mean'],g_blk[g_blk['Player_ID']==playerID]['std'],1))))
            tov = (round(int(np.random.normal(g_tov[g_tov['Player_ID']==playerID]['mean'],g_tov[g_tov['Player_ID']==playerID]['std'],1))))
            score = pts + round(1.7*reb) + round(2*ass) + round(9*stl)+ round(6*blk)-round(5*tov)
            if score >= 0:
                return score
            else:
                return 0
        sim += 1

In [326]:
def sim_team_mvp(playerIDs, numSims=1):
    '''
    Takes in player IDs to pass through to player_sims function.
    Returns points scored. Can keep track of a list of player IDs
    which is a team.
    '''
    sims_raw = []
    for playerID in playerIDs:
        player_sims = sim_player_mvp(playerID, numSims)
        sims_raw.append(player_sims)
    return sims_raw

In [327]:
def sim_games_mvp(team1, team2, numSims=1):
    '''
    Takes in two teams and passes them through the sim team function 
    and returns how many games each team won. Uses get_roster_ids to get player
    IDs to pass through. Modified for mvp
    '''
    team1_roster = get_roster_ids(team1)
    team2_roster = get_roster_ids(team2)
    results = [0,0]
    for game in range(0, numSims):
        team1_sims = sim_team_mvp(team1_roster, 1)
        team2_sims = sim_team_mvp(team2_roster, 1)
        if sum(team1_sims) > sum(team2_sims):
            results[0] += 1
        else:
            results[1] += 1
    return results

#after trade deadline
def sim_games2_mvp(team1, team2, numSims=1):
    '''
    Takes in two teams and passes them through the sim team function 
    and returns how many games each team won. Modified for mvp.
    '''
    team1_roster = get_roster_ids2(team1)
    team2_roster = get_roster_ids2(team2)
    results = [0,0]
    for game in range(0, numSims):
        team1_sims = sim_team_mvp(team1_roster, 1)
        team2_sims = sim_team_mvp(team2_roster, 1)
        if sum(team1_sims) > sum(team2_sims):
            results[0] += 1
        else:
            results[1] += 1
    return results

In [328]:
def sim_season_mvp(numSimss=1):
    '''
    Takes in how many seasons you want to run and returns a dataframe
    of the different seasons and each team's number of wins in that season. 
    Modified to discard MVP through player scoring function
    '''
    team_wins = pd.DataFrame(columns=[team_list])
    team_wins = team_wins.append(pd.Series(name='Win_Count'))
    team_wins.fillna(0,inplace=True)
    
    for games in range(len(gameid_list)):
        wins = []
        if gameid_list[games] == pretradegames['GAME_ID'].iloc[games]:
            wins = sim_games_mvp(get_game(games)[0],get_game(games)[1],numSimss)
            if wins[0] > wins[1]:
                team_wins[get_game(games)[0]] +=1
            else:
                team_wins[get_game(games)[1]] +=1
        else:
            wins = sim_games2_mvp(get_game(games)[0],get_game(games)[1],numSimss)
            if wins[0] > wins[1]:
                team_wins[get_game(games)[0]] +=1
            else:
                team_wins[get_game(games)[1]] +=1
    return team_wins

In [459]:
def mvp_sim(player_id, b):
    sim_list = []
    mvp_player = []
    player = player_id
    for i in range(b):
        sim_list.append('sim'+str(i))
    for simulation in sim_list:
        simulation = sim_season_mvp(1)
        simulation = simulation.transpose()
        simulation = simulation.reset_index()
        mvp_player.append(simulation)
    mvp_player_df = functools.reduce(lambda  left,right: pd.merge(left,right,on=['level_0'],
                                            how='outer'), mvp_player)
    return mvp_player_df

In [344]:
#list of mvp players with their names
#ran each simulation below, analysis on sim_results notebook
mvp_list = ['giannis','harden','george','jokic','curry','lillard','embiid','kawhi','gobert','lebron']
mvp = [203507,201935,202331,203999,201939,203081,203954,202695,203497,2544]

In [460]:
player=203507
giannas_df = mvp_sim(203507,10)

In [462]:
player=201935
harden_df = mvp_sim(201935,10)

In [463]:
player=202331
george_df = mvp_sim(202331,10)

In [464]:
player=203999
jokic_df = mvp_sim(203999,10)

In [465]:
player=201939
curry_df = mvp_sim(201939,10)

In [466]:
player=203081
lillard_df = mvp_sim(203081,10)

In [467]:
player=203954
embiid_df = mvp_sim(203954,10)

In [468]:
player=202695
kawhi_df = mvp_sim(202695,10)

In [469]:
player=203497
gobert_df = mvp_sim(203497,10)

In [470]:
player=2544
lebron_df = mvp_sim(2544,10)

In [471]:
giannas_df.to_csv('giannas_df.csv')
harden_df.to_csv('harden_df.csv')
george_df.to_csv('george_df.csv')
jokic_df.to_csv('jokic_df.csv')
curry_df.to_csv('curry_df.csv')
lillard_df.to_csv('lillard_df.csv')
embiid_df.to_csv('embiid_df.csv')
kawhi_df.to_csv('kawhi_df.csv')
gobert_df.to_csv('gobert_df.csv')
lebron_df.to_csv('lebron_df.csv')